In [1]:
library(benchmarkme)
get_platform_info()$OS.type
get_r_version()$version.string
get_cpu()$model_name;get_cpu()$no_of_cores
get_ram()

[1] "windows"

[1] "R version 3.5.1 (2018-07-02)"

[1] "Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz"

[1] 8

8.59 GB

In [2]:
## Package was modified to allow complete cross mating
#library("devtools")
#install_github("gglinzijie/xbreed")
library("xbreed")

("|-----------------------------------------------------|")
("|                      xbreed                         |")
("|    Genomic simulation of purebreds and crossbreds   |")
("|               March 2017 Version 1.0.1              |")
("|                                                     |")
("|             H.Esfandyari,A.C.Sorensen               |")
("| Center for Quantitative Qenetics and Genomics (QGG) |")
("|             Aarhus University,Denmark               |")
("|                                                     |")
("|-----------------------------------------------------|")
("|Questions and bugs: esfandyari.hadi@gmail.com        |")
("|Development of xbreed was supported by GenSAP.       |")
("|-----------------------------------------------------|")


# Genome specification

In [3]:
#Number of markers per chr from chapter 1
lin_map<-read.table("raw.map")
m=1:22
for(i in 1:22) {m[i]=dim(lin_map[lin_map$V1==i,])[1]}
sum(m)
#nf <- layout(1,widths=3, heights=4, TRUE)
#barplot(m,col = "cadetblue1",ylim=c(0,400),xlim=c(0,23),cex.axis = 1.5,names.arg = )

[1] 3928

In [4]:
#data from chaptre 1
geno<-read.table("Hetero_realigned_cov10_filtered3.raw")
pheno<-read.csv("2017heteroPheno.csv", header=T)
attach(pheno)

In [5]:
#parameter of genome
no.chr<-22
genome<-data.frame(matrix(NA, nrow=no.chr, ncol=6))
names(genome)<-c("chr","len","nmrk","mpos","nqtl","qpos")
genome$chr<-c(1:no.chr) #Chromosome id from 1 to 22
genome$len<-c(200,rep(100,21))#Chromosome length in cM
genome$nmrk<-c(m) #Number of markers, 3928 in total 
genome$mpos<-c('even') 
genome$nqtl<-c(40) #Number of qtl  40*22 = 880 in total
genome$qpos<-c('rnd')

# Historiacal population

In [6]:
#Historical population 
hp<-make_hp(hpsize=1000 ,ng=5000,h2=0.654,d2=0,phen_var=84,
            genome=genome,mutr=2.5*10**-4,laf=1)

---sel_seq_qtl is missing, it has been set to default value of 0
---sel_seq_mrk is missing, it has been set to default value of 0
Historical pop is initialized...
Simulating trait ...
Output data preparation ...
Establishment of historical population completed


In [7]:
#validation
mutr<-2.5*10**-4
ne<-1000
k<-2 
Fneu<-4*ne*mutr
(Expected_het1<-Fneu/(1+Fneu))
(Expected_het1<-1-((1+((Fneu)/(k-1)))/(1+((Fneu*k)/(k-1)))))
(het_observed<-mean(2*(hp$freqMrk[,3]*hp$freqMrk[,4])))

[1] 0.5

[1] 0.3333333

[1] 0.3981657

In [8]:
Male_founders<-data.frame(number=10,select='rnd')
Female_founders<-data.frame(number=10,select='rnd')

In [9]:
Selection<-data.frame(matrix(NA, nrow=2, ncol=3))
names(Selection)<-c('Number','type','Value') 
Selection$Number[1:2]<-c(10,10)
Selection$type[1:2]<-c('rnd','rnd')
Selection$Value[1:2]<-c('l','l')

In [10]:
sh_output<-data.frame(matrix(NA, nrow=1, ncol=4))
names(sh_output)<-c("data","qtl","freq_mrk","marker")
sh_output[1]<-c(1) 
sh_output[2]<-c(1) 
sh_output[3]<-c(1)
sh_output[4]<-c(1)

In [11]:
#10 sires and 10 dams perform complete cross; 200 eggs were produced per dam.
RP<-sample_hp(hp_out=hp,Male_founders= Male_founders,
              Female_founders=Female_founders,ng=1,Selection=Selection, 
              litter_size=30,saveAt="SNP3928",sh_output=sh_output,Display=FALSE)

Controlling input data ...
Intializing base population ...
Generation 0 started ......... 
Generation 0 is finished. Time taken: 23.10031
Generation 1 started ......... 
Generation 1 is finished. Time taken: 29.10541
Output data preparation ...
Writing output files ...
Sampling hp is done!


In [12]:
#function for calculate the allele coding, which is 0, 1, 2
bin_snp<-function(mat){
s1<-seq(1,ncol(mat),2)
s2<-seq(2,ncol(mat),2)
a1<-mat[,s1]+mat[,s2]
a1[a1==3]=1
a1[a1==4]=0
snp_code<-a1
return(snp_code)
 }

In [17]:
##geno and pheno from simulated data
pheno<-RP$output[[2]]$data$phen
tbvp<-RP$output[[2]]$data$tbvp
n<-bin_snp(RP$output[[2]]$mrk[,3:7858])
x<-as.matrix(n)-1
colnames(x)<-1:3928
row.names(x)<-1:3000

In [19]:
write.table(x,"2.3_geno.txt")
write.table(pheno,"2.3_pheno.txt")
write.table(tbvp,"2.3_tbvp.txt")

In [2]:
# Simulation result
x<-as.matrix(read.table("2.3_geno.txt"))
pheno<-as.numeric(unlist(read.table("2.3_pheno.txt")))
tbvp<-as.numeric(unlist(read.table("2.3_tbvp.txt")))

In [3]:
##load packages
library(doParallel)
library(foreach)
cl<-makeCluster(8) 
repeats <- 10
n.fold <- 5 
acc<-list()
library(rrBLUP)
packageVersion("rrBLUP") 

Loading required package: foreach
Loading required package: iterators
Loading required package: parallel


[1] '4.6'

In [44]:
cal_se<-function(acc){sd(acc)/sqrt(repeats-1)}

In [45]:
#function for calculating the accuracy of GP with varying size of reference population
cal_acc<-function(x,n.sample){
set.seed(100)
id<-sample(1:dim(x)[1],n.sample)
#relationship matrix (Endelman at al. 2011)
A <- A.mat(x[id,], n.core=8)
row.names(A)=1:n.sample;colnames(A)=1:n.sample
data <- data.frame(tbcw=pheno[id],tbvp=tbvp[id],gid=1:n.sample)
result<-data.frame(heritability=rep(NA,50),
                   bias_r=rep(NA,50),
                   unbias_r=rep(NA,50),
                   true_r=rep(NA,50))                   
registerDoParallel(cl)
foreach(j=1:repeats,.combine = "rbind") %do% {
        set.seed(100+3*j+1)
        id <- sample(1:n.sample %% n.fold) + 1 
        foreach(i=1:n.fold,.packages="rrBLUP") %do% {
         bcw_test <- data
         bcw_test$tbcw[id == i] <- NA
         res <- kin.blup(bcw_test, K=A, geno="gid", pheno="tbcw")
         result[5*(j-1)+i,1]<-res$Vg/(res$Vg+res$Ve)
         result[5*(j-1)+i,2]<-cor(data$tbcw[id==i],res$pred[id==i])
         result[5*(j-1)+i,3]<-cor(data$tbcw[id==i],res$pred[id==i])/sqrt(res$Vg/(res$Vg+res$Ve))
         result[5*(j-1)+i,4]<-cor(data$tbvp[id==i],res$pred[id==i])
    }
   }
stopImplicitCluster()
return(c(apply(result, 2, mean),apply(result,2,cal_se)))    
}

In [46]:
n.sample<-c(250,500,750,1000,1250,1500,1750,2000)
all_acc<-matrix(NA,nrow = 8,ncol = length(n.sample))
all_accBA<-all_accBB<-all_accBC<-all_accBL<-all_acc

In [47]:
#implemente the caculation 
for (i in 1:length(n.sample)){
    m<-n.sample[i]
    all_acc[,i]<-cal_acc(x,m)}

In [70]:
colnames(all_acc)<-n.sample
row.names(all_acc)<-c("h2","bias_r","unbias_r","true_r","h2_se","bias_r_se","unbias_r_se","true_r_se")

In [51]:
library("BGLR")
packageVersion("BGLR") 

[1] '1.0.5'

In [53]:
cal_accB<-function(x,n.sample,M){
set.seed(100)
id<-sample(1:dim(x)[1],n.sample)
x1<-x[id,]
data <- data.frame(tbcw=pheno[id],tbvp=tbvp[id],gid=1:n.sample)
result<-data.frame(heritability=rep(NA,50),
                   bias_r=rep(NA,50),
                   unbias_r=rep(NA,50),
                   true_r=rep(NA,50))                   
registerDoParallel(cl)
foreach(j=1:repeats,.combine = "rbind") %do% {
        set.seed(100+3*j+1)
        id <- sample(1:n.sample %% n.fold) + 1 
        foreach(i=1:n.fold,.packages="BGLR") %do% {
         bcw_test <- data
         bcw_test$tbcw[id == i] <- NA
         fm=BGLR(y=bcw_test$tbcw,ETA=list(list(X=x1,model=M,saveEffects=T,saveAt='fm_')),nIter=2000,burnIn=1000,verbose=F)
         varU=var(x%*%fm$ETA[[1]]$b)
         varE=fm$varE
         h2=varU/(varU+varE)
         result[5*(j-1)+i,1]<-h2
         result[5*(j-1)+i,2]<-cor(data$tbcw[id == i],fm$yHat[id == i])
         result[5*(j-1)+i,3]<-cor(data$tbcw[id==i],fm$yHat[id == i])/sqrt(h2)
         result[5*(j-1)+i,4]<-cor(data$tbvp[id==i],fm$yHat[id == i])
    }
   }
stopImplicitCluster()
return(c(apply(result, 2, mean),apply(result,2,cal_se)))    
}

In [54]:
for (i in 1:length(n.sample)){all_accBA[,i]<-cal_accB(x,n.sample[i],"BayesA")}

In [55]:
for (i in 1:length(n.sample)){all_accBB[,i]<-cal_accB(x,n.sample[i],"BayesB")}

In [56]:
for (i in 1:length(n.sample)){all_accBC[,i]<-cal_accB(x,n.sample[i],"BayesC")}

In [ ]:
for (i in 1:length(n.sample)){all_accBL[,i]<-cal_accB(x,n.sample[i],"BL")}

In [58]:
colnames(all_accBA)<-n.sample
colnames(all_accBB)<-n.sample
colnames(all_accBC)<-n.sample
colnames(all_accBL)<-n.sample
row.names(all_accBA)<-c("h2","bias_r","unbias_r","true_r","h2_se","bias_r_se","unbias_r_se","true_r_se")
row.names(all_accBB)<-c("h2","bias_r","unbias_r","true_r","h2_se","bias_r_se","unbias_r_se","true_r_se")
row.names(all_accBC)<-c("h2","bias_r","unbias_r","true_r","h2_se","bias_r_se","unbias_r_se","true_r_se")
row.names(all_accBL)<-c("h2","bias_r","unbias_r","true_r","h2_se","bias_r_se","unbias_r_se","true_r_se")

In [68]:
(sum<-list(GBLUP=all_acc,
                BA=all_accBA,
                BB=all_accBB,
                BC=all_accBC,
                BL=all_accBL))

,250,500,750,1000,1250,1500,1750,2000
h2,0.61374649,0.66151828,0.62638003,0.589046184,0.590205106,0.538965250,0.550585342,0.553422955
bias_r,0.52615287,0.59066718,0.61311733,0.612229382,0.622635211,0.618401039,0.627044919,0.624070602
unbias_r,0.67766917,0.72842492,0.77611581,0.798653743,0.811682369,0.843208484,0.845964760,0.839417178
true_r,0.62847095,0.70519204,0.73593335,0.742633087,0.759736091,0.770348655,0.778467888,0.781107037
h2_se,0.01914803,0.01341505,0.01046737,0.009032739,0.008742087,0.007466956,0.007552235,0.006485728
bias_r_se,0.03272688,0.01757305,0.01411353,0.009910763,0.012712881,0.009400992,0.010754479,0.007665713
unbias_r_se,0.04981851,0.02700728,0.02202480,0.016303351,0.020552535,0.016156245,0.017848986,0.012572302
true_r_se,0.02895124,0.01652853,0.01113805,0.008747125,0.009285673,0.007661239,0.006509593,0.005460806
,250,500,750,1000,1250,1500,1750,2000
h2,0.45848071,0.53472558,0.52430267,0.501658040,0.517084160,0.483007638,0.494474698,0.496579539


In [71]:
#save to xlsx file
library("xlsx")
write.xlsx(sum, "2_3_result.xlsx") 

In [76]:
library("readxl")
plot<-read_excel("2_3_result.xlsx")